In [2]:
# This is the most recent and important file - 2 versions evolved due to moving an open file 
# Run in a sequence 1, 2 for landscape then 3 and 4 for portrait - need to be integrated but functioning now 
# Careful as tricky to get annotations scaled correctly when converting from different aspect ratios 
# Handles PNGs and Jpegs. All images go out in the same format they came in. No inadvertent conversion to JPEG. 
# Copies pairs where the image already meets the target size. 

import os
import cv2
import numpy as np
import shutil 


# 1. Run for landscape images to resize and pad evenly on the top and bottom to reach the target size (to match the vide frames)
def resize_and_crop_landscape_crop_even(input_folder, output_folder, target_width, max_height):
    """
    Resize landscape images to a target width and evenly crop the top and bottom 
    if the height exceeds the maximum. Adjust annotations accordingly.
    """
    print("Starting resizing and cropping (even) for landscape images...")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            annotation_path = os.path.splitext(input_path)[0] + ".txt"
            output_annotation_path = os.path.splitext(output_path)[0] + ".txt"

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading {input_path}. Skipping.")
                continue

            original_height, original_width = image.shape[:2]

            # Handle images already at target size
            if original_width == target_width and original_height == max_height:
                print(f"Image already at target size: {file_name}. Copying without changes.")
                shutil.copy(input_path, output_path)  # Copy image
                if os.path.exists(annotation_path):  # Copy annotation if it exists
                    shutil.copy(annotation_path, output_annotation_path)
                continue

            # Skip non-landscape images
            if original_width <= original_height:
                print(f"Skipping non-landscape image: {file_name}")
                continue

            # Resize to target width while maintaining aspect ratio
            scale_ratio = target_width / original_width
            new_height = int(original_height * scale_ratio)
            resized_image = cv2.resize(image, (target_width, new_height))

            # Crop height evenly from top and bottom if it exceeds the maximum
            crop_offset = 0
            if new_height > max_height:
                crop_offset = (new_height - max_height) // 2  # Calculate crop offset
                resized_image = resized_image[crop_offset:crop_offset + max_height, :]  # Crop evenly

            # Determine file format
            file_extension = ".png" if file_name.lower().endswith(".png") else ".jpg"
            output_path = os.path.splitext(output_path)[0] + file_extension

            # Save the processed image
            cv2.imwrite(output_path, resized_image)
            print(f"Resized and cropped (even): {output_path}")

    print("Completed resizing and cropping (even) for landscape images.")

# 2. Run to adjust the annotations after the images are resized - see not below re future integration  
def adjust_annotations_for_folder_landscape(input_annotation_folder, input_image_folder, output_annotation_folder, target_width, target_height):
    """
    Adjust YOLO annotations for all landscape images in a folder and save them for a batch test case.

    Args:
        input_annotation_folder (str): Folder containing annotation files.
        input_image_folder (str): Folder containing image files.
        output_annotation_folder (str): Folder to save adjusted annotation files.
        target_width (int): Target width for resized images.
        target_height (int): Target height for resized images.
    """
    # Ensure output folder exists
    if not os.path.exists(output_annotation_folder):
        os.makedirs(output_annotation_folder)

    # Process each annotation file
    for annotation_file in os.listdir(input_annotation_folder):
        if annotation_file.endswith('.txt'):
            annotation_path = os.path.join(input_annotation_folder, annotation_file)
            #image_path = os.path.join(input_image_folder, annotation_file.replace('.txt', '.jpg

            # Check for both .jpg and .png images
            image_path_jpg = os.path.join(input_image_folder, annotation_file.replace('.txt', '.jpg'))
            image_path_png = os.path.join(input_image_folder, annotation_file.replace('.txt', '.png'))
            image_path = image_path_jpg if os.path.exists(image_path_jpg) else image_path_png
            
            # Verify the image file exists
            if not os.path.exists(image_path):
                print(f"Image file not found for annotation: {annotation_file}. Skipping.")
                continue

            output_annotation_path = os.path.join(output_annotation_folder, annotation_file)

            # Get image dimensions
            image = cv2.imread(image_path)
            original_height, original_width = image.shape[:2]

            # Skip portrait images
            if original_width <= original_height:
                print(f"Skipping portrait image: {annotation_file}")
                continue

            # Calculate scale
            scale_x = target_width / original_width
            scale_y = target_width / original_width  # Maintain aspect ratio
            new_height = int(original_height * scale_x)

            # Corrected crop offset calculation
            crop_offset = 0
            if new_height > target_height:
                crop_offset = (new_height - target_height) // 2
            print(f"Processing {annotation_file}: crop_offset={crop_offset}")

            adjusted_lines = []

            # Read annotations
            with open(annotation_path, 'r') as file:
                annotations = file.readlines()

            for line in annotations:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())

                # Scale coordinates and dimensions
                x_center_scaled = x_center * original_width * scale_x
                y_center_scaled = y_center * original_height * scale_y
                width_scaled = width * original_width * scale_x
                height_scaled = height * original_height * scale_y

                # Apply crop offset
                y_center_scaled -= crop_offset

                # Clamp to image bounds
                x_left = max(0, x_center_scaled - width_scaled / 2)
                x_right = min(target_width, x_center_scaled + width_scaled / 2)
                y_top = max(0, y_center_scaled - height_scaled / 2)
                y_bottom = min(target_height, y_center_scaled + height_scaled / 2)

                # Recalculate dimensions
                width_clamped = max(0, x_right - x_left)
                height_clamped = max(0, y_bottom - y_top)

                if width_clamped > 0 and height_clamped > 0:
                    x_center_normalized = (x_left + x_right) / (2 * target_width)
                    y_center_normalized = (y_top + y_bottom) / (2 * target_height)
                    width_normalized = width_clamped / target_width
                    height_normalized = height_clamped / target_height

                    # Append corrected annotation
                    adjusted_lines.append(
                        f"{int(class_id)} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"
                    )
                else:
                    print(f"Bounding box for class {class_id} is invalid after clamping: skipping.")

            # Save adjusted annotations
            with open(output_annotation_path, 'w') as file:
                file.write('\n'.join(adjusted_lines))

            print(f"Adjusted annotations saved to: {output_annotation_path}")

# 3. resize and pad portrait images 
def resize_and_pad_portrait(input_folder, output_folder, target_width, target_height):
    """
    Resize and pad portrait images and adjust annotations.
    """
    print("Starting resizing and padding for portrait images...")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            annotation_path = os.path.splitext(input_path)[0] + ".txt"
            output_annotation_path = os.path.splitext(output_path)[0] + ".txt"

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading {input_path}. Skipping.")
                continue

            original_height, original_width = image.shape[:2]

            # Handle images already at target size
            if original_width == target_width and original_height == target_height:
                print(f"Image already at target size: {file_name}. Copying without changes.")
                shutil.copy(input_path, output_path)  # Copy image
                if os.path.exists(annotation_path):  # Copy annotation if it exists
                    shutil.copy(annotation_path, output_annotation_path)
                continue

            # Skip landscape images
            if original_width > original_height:
                print(f"Skipping landscape image: {file_name}")
                continue

            # Calculate scaling factors
            scale = min(target_width / original_width, target_height / original_height)
            new_width = int(original_width * scale)
            new_height = int(original_height * scale)

            # Resize the image
            resized_image = cv2.resize(image, (new_width, new_height))

            # Create a blank canvas
            padded_image = np.zeros((target_height, target_width, 3), dtype=np.uint8)
            # Center the resized image on the canvas
            y_offset = (target_height - new_height) // 2
            x_offset = (target_width - new_width) // 2
            padded_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

            # Determine file format
            file_extension = ".png" if file_name.lower().endswith(".png") else ".jpg"
            output_path = os.path.splitext(output_path)[0] + file_extension

            # Save the processed image
            cv2.imwrite(output_path, padded_image)
            print(f"Resized and padded: {output_path}")

    print("Completed resizing and padding for portrait images.")

# 4. Adjust annotations after resizing portrait images 
def adjust_annotations_for_portrait_folder(input_annotation_folder, input_image_folder, output_annotation_folder, target_width, target_height):
    """
    Adjust YOLO annotations for all portrait images in a folder.

    Args:
        input_annotation_folder (str): Folder containing annotation files.
        input_image_folder (str): Folder containing image files.
        output_annotation_folder (str): Folder to save adjusted annotation files.
        target_width (int): Target width for resized images.
        target_height (int): Target height for resized images.
    """
    # Ensure output folder exists
    if not os.path.exists(output_annotation_folder):
        os.makedirs(output_annotation_folder)

    for annotation_file in os.listdir(input_annotation_folder):
        if annotation_file.endswith('.txt'):
            annotation_path = os.path.join(input_annotation_folder, annotation_file)

            # Check for both .jpg and .png images
            image_path_jpg = os.path.join(input_image_folder, annotation_file.replace('.txt', '.jpg'))
            image_path_png = os.path.join(input_image_folder, annotation_file.replace('.txt', '.png'))
            image_path = image_path_jpg if os.path.exists(image_path_jpg) else image_path_png

            # Verify the image file exists
            if not os.path.exists(image_path):
                print(f"Image file not found for annotation: {annotation_file}. Skipping.")
                continue

            #image_path = os.path.join(input_image_folder, annotation_file.replace('.txt', '.jpg'))
            output_annotation_path = os.path.join(output_annotation_folder, annotation_file)

            # Get image dimensions
            image = cv2.imread(image_path)
            original_height, original_width = image.shape[:2]

            # Skip landscape images
            if original_width > original_height:
                print(f"Skipping landscape image: {annotation_file}")
                continue

            # Calculate scaling factors and padding
            scale = target_height / original_height
            new_width = int(original_width * scale)
            x_padding = (target_width - new_width) // 2  # Padding for centering the image horizontally

            # Check if padding is valid
            if new_width > target_width:
                print(f"Error: Resized width ({new_width}) exceeds target width ({target_width}). Skipping.")
                continue

            print(f"Processing {annotation_file}: x_padding={x_padding}, scale={scale}")

            adjusted_lines = []

            # Read annotations
            with open(annotation_path, 'r') as file:
                annotations = file.readlines()

            for line in annotations:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())

                # Scale coordinates and dimensions
                x_center_scaled = x_center * original_width * scale + x_padding
                y_center_scaled = y_center * original_height * scale
                width_scaled = width * original_width * scale
                height_scaled = height * original_height * scale

                # Clamp to image bounds
                x_left = max(0, x_center_scaled - width_scaled / 2)
                x_right = min(target_width, x_center_scaled + width_scaled / 2)
                y_top = max(0, y_center_scaled - height_scaled / 2)
                y_bottom = min(target_height, y_center_scaled + height_scaled / 2)

                # Recalculate dimensions
                width_clamped = max(0, x_right - x_left)
                height_clamped = max(0, y_bottom - y_top)

                if width_clamped > 0 and height_clamped > 0:
                    x_center_normalized = (x_left + x_right) / (2 * target_width)
                    y_center_normalized = (y_top + y_bottom) / (2 * target_height)
                    width_normalized = width_clamped / target_width
                    height_normalized = height_clamped / target_height

                    # Append corrected annotation
                    adjusted_lines.append(
                        f"{int(class_id)} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"
                    )
                else:
                    print(f"Bounding box for class {class_id} is invalid after clamping: skipping.")

            # Save adjusted annotations
            with open(output_annotation_path, 'w') as file:
                file.write('\n'.join(adjusted_lines))

            print(f"Adjusted annotations saved to: {output_annotation_path}")


###################################################################################################################

# Paths and target dimensions
#input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_145"
#output_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_146R1"
#input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151"
input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151"
output_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final"
target_width = 1920
target_height = 1080

# Test the function single file - For testing only 
#annotation_path = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_146/Job_146_08758.txt'
#image_path = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_146/Job_146_08758.jpg'
#output_path = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_146R1/DSC08771.txt'

# 1. Resize and crop landscape images in the source folder 
resize_and_crop_landscape_crop_even(input_folder, output_folder, target_width, target_height)

# Update annotations for landscape  images in a a folader
# 2. 
adjust_annotations_for_folder_landscape(input_folder, input_folder, output_folder, target_width, target_height)

# 3.  Resize and pad the PORTRAIT images 
#resize_and_pad_portrait(input_folder, output_folder, target_width, target_height)

# 4. Adjust annotations for all portrait images in the folder - Skips landscape 
#adjust_annotations_for_portrait_folder(input_folder, input_folder, output_folder, target_width, target_height)

# To do:
# Does not work for some image size where image is smaller than the target size - image gets resized but annotation notcreated - skipped 
# For now add the annotation manually
# Reun separately for portrait and landscape and merge 

###########################################################################################################
# Sample output:
#Starting resizing and cropping (even) for landscape images...
#Image already at target size: Job_151_000001.jpg. Copying without changes.
#Image already at target size: Job_151_000002.png. Copying without changes.
#Image already at target size: Job_151_000003.jpg. Copying without changes.
#Image already at target size: Job_151_000005.png. Copying without changes.
#Image already at target size: Job_151_000006.png. Copying without changes.
#Image already at target size: Job_151_000010.jpg. Copying without changes.
#Completed resizing and cropping (even) for landscape images.
#Processing Job_151_000001.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000001.txt
#Processing Job_151_000003.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000003.txt
#Processing Job_151_000005.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000005.txt
#Processing Job_151_000002.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000002.txt
#Processing Job_151_000006.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000006.txt
#Processing Job_151_000010.txt: crop_offset=0
#Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000010.txt

Starting resizing and cropping (even) for landscape images...
Image already at target size: Job_151_000001.jpg. Copying without changes.
Image already at target size: Job_151_000002.png. Copying without changes.
Image already at target size: Job_151_000003.jpg. Copying without changes.
Image already at target size: Job_151_000005.png. Copying without changes.
Image already at target size: Job_151_000006.png. Copying without changes.
Image already at target size: Job_151_000010.jpg. Copying without changes.
Completed resizing and cropping (even) for landscape images.
Processing Job_151_000001.txt: crop_offset=0
Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000001.txt
Processing Job_151_000003.txt: crop_offset=0
Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151Final\Job_151_000003.txt
Processing Job_151_000005.txt: crop_offset=0
Adjusted annotations saved to: D:/FlagDetectionDatasets/ExportedDa